In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import glob
import os

A utilização de cada uma das bibliotecas acima:
pandas: Pegar os dados dos csv's.
numpy: ?
mathplotlib e seaborn: Para geração de gráficos estáticos.
glob: Biblioteca que ajuda a identificar arquivos cujo os nomes seguem um padrão, no nosso caso o .csv.
os: Ajuda a navegar pelos caminhos

In [ ]:
pastas = 'data_raw'
pastas_zip = glob.glob(os.path.join(pastas, "*.zip"))
